In [1]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, SeparableConv2D
from keras.layers import concatenate, add, Lambda
from keras.layers import TimeDistributed
from keras.layers import LSTM
from keras.models import Model
from keras.regularizers import l2

from keras.applications import imagenet_utils, Xception
from keras.preprocessing.image import img_to_array
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py
import random

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
from helpers import TrainingMonitor
from helpers import Utils

In [3]:
output_path = "../output/"

In [4]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [5]:
db = h5py.File("../input/datasets/UCF-101-hdf5/fixed-sequance-rgbmean.hdf5")
x = db["images"][:]
y = db["labels"][:]

In [6]:
train_split, val_split = 0.7, 0.15

train_len = int(train_split * x.shape[0])
train_val_len = int((train_split + val_split) * x.shape[0])

x_train = x[:train_len]
x_val = x[train_len:train_val_len]
x_test = x[train_val_len:]

y_train = y[:train_len]
y_val = y[train_len:train_val_len]
y_test = y[train_val_len:]

In [7]:
num_classes = y_test.shape[1]
num_classes

8

In [8]:
x_train.shape[2:]

(60, 80, 3)

In [9]:
cnn = Sequential() 
cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(60, 80, 3)))
cnn.add(MaxPooling2D(pool_size=(2, 2))) 
cnn.add(Flatten())

In [10]:
cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 58, 78, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 39, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 72384)             0         
Total params: 1,792
Trainable params: 1,792
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model = Sequential()
model.add(TimeDistributed(cnn, input_shape=(20, 60, 80, 3)))
model.add(LSTM(256)) 
model.add(Dense(num_classes, activation="softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train,
            batch_size=4,
            epochs=5,
            validation_data=(x_val, y_val),
            shuffle="batch")

Train on 140 samples, validate on 30 samples
Epoch 1/5


In [ ]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )